In [1]:
path = r'C:/Users/ruw/Desktop'
import sys # https://stackoverflow.com/questions/4383571/importing-files-from-different-folder
sys.path.insert(1, rf'{path}\RA\penicilin\Libaries')



In [2]:
matching_database = "ev391cutoff"
database = "penicillin_cut_off"

In [27]:
import reload_lib as rl
# import lca
import standards as s
import sensitivity as st
import brightway2 as bw 

import bw2data as bd
import bw2calc as bc
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt


import standards as s
import database_manipulation as dm
import lcia_results as lr


In [7]:
from lca import LCA

lca_init = LCA(path=path, matching_database=matching_database)


In [42]:
ics = lca_init.lcia_impact_method()
data = lca_init.LCA_initialization()
fu = data["Penicillin G, defined system"]
fu

Initialization is completed for penicillin_cut_off_IN


{'production of alchohol wipes' (unit, DK, None): 1,
 'packaging of glass vials with penicillin G' (unit, IN, None): 0.1,
 'production of a pair of gloves' (unit, MY, None): 1,
 'treatment of hazardous waste, hazardous waste incineration, with energy recovery' (kilogram, CH, None): -0.09551460666666667,
 'production of IV bag' (unit, RER, None): 1,
 'medical connector' (unit, GLO, None): 1,
 'market for sodium chlorate, powder' (kilogram, RER, None): 0.0009000000000000001,
 'stopcock' (unit, GLO, None): 1,
 'market for water, ultrapure' (kilogram, RER, None): 0.1,
 'waste packaging paper, Recycled Content cut-off' (kilogram, GLO, None): 0.020203974}

In [86]:
db = lca_init.db
func_unit = []
idx_lst = []
func_unit_keys = []
for act in db:
    if "defined system" in act["name"]:
        idx_lst.append(act["name"])
        func_unit_keys.append(act)
func_unit_keys.sort()
func_unit_keys.reverse()
func_unit_keys
idx_lst.sort()

for key in func_unit_keys:
    func_unit.append({key : 1})

print(idx_lst[0], func_unit_keys[0])

Penicillin G, defined system 'Penicillin G, defined system' (unit, DK, None)


In [106]:
db = lca_init.db
func_unit = []
idx_lst = []
for act in db:
    if "defined system" in act["name"] and "G" in act["name"]:

        for exc in act.exchanges():
            if "techno" in exc["type"]:
                func_unit.append({exc.input : exc["amount"]})
                idx_lst.append(exc.input["name"])
func_unit


[{'production of alchohol wipes' (unit, DK, None): 1},
 {'packaging of glass vials with penicillin G' (unit, IN, None): 0.1},
 {'production of a pair of gloves' (unit, MY, None): 1},
 {'treatment of hazardous waste, hazardous waste incineration, with energy recovery' (kilogram, CH, None): -0.09551460666666667},
 {'production of IV bag' (unit, RER, None): 1},
 {'medical connector' (unit, GLO, None): 1},
 {'market for sodium chlorate, powder' (kilogram, RER, None): 0.0009000000000000001},
 {'stopcock' (unit, GLO, None): 1},
 {'market for water, ultrapure' (kilogram, RER, None): 0.1},
 {'waste packaging paper, Recycled Content cut-off' (kilogram, GLO, None): 0.020203974}]

In [ ]:
# Initialize a DataFrame to store sensitivity results

# Set up the calculation environment for sensitivity analysis
bd.calculation_setups['1 treatment'] = {'inv': func_unit["Penicillin G, defined system"], 'ia': ics}

# Perform the MultiLCA calculation
res = bc.MultiLCA('1 treatment')
results_array = res.results

# # Store results in the DataFrame
# for idx, res in enumerate(sens_results_array):
#     df_sens.iat[idx, 0] = res

In [54]:
results_array
df = pd.DataFrame(0, index=idx_lst["Penicillin G, defined system"], columns=ics, dtype=object)
df

,"(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, acidification: terrestrial, terrestrial acidification potential (TAP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, climate change, global warming potential (GWP1000))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, ecotoxicity: freshwater, freshwater ecotoxicity potential (FETP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, ecotoxicity: marine, marine ecotoxicity potential (METP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, ecotoxicity: terrestrial, terrestrial ecotoxicity potential (TETP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, energy resources: non-renewable, fossil, fossil fuel potential (FFP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, eutrophication: freshwater, freshwater eutrophication potential (FEP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, eutrophication: marine, marine eutrophication potential (MEP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, human toxicity: carcinogenic, human toxicity potential (HTPc))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, human toxicity: non-carcinogenic, human toxicity potential (HTPnc))",...,"(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, land use, agricultural land occupation (LOP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, material resources: metals/minerals, surplus ore potential (SOP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, ozone depletion, ozone depletion potential (ODPinfinite))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, particulate matter formation, particulate matter formation potential (PMFP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, photochemical oxidant formation: human health, photochemical oxidant formation potential: humans (HOFP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, photochemical oxidant formation: terrestrial ecosystems, photochemical oxidant formation potential: ecosystems (EOFP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, water use, water consumption potential (WCP))","(ReCiPe 2016 v1.03, endpoint (H) - no biogenic, total: ecosystem quality, ecosystem quality)","(ReCiPe 2016 v1.03, endpoint (H) - no biogenic, total: human health, human health)","(ReCiPe 2016 v1.03, endpoint (H) - no biogenic, total: natural resources, natural resources)"
production of alchohol wipes,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
packaging of glass vials with penicillin G,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
production of a pair of gloves,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"treatment of hazardous waste, hazardous waste incineration, with energy recovery",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
production of IV bag,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
medical connector,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"market for sodium chlorate, powder",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
stopcock,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"market for water, ultrapure",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"waste packaging paper, Recycled Content cut-off",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
# Store results in the DataFrame
for idx, res in enumerate(results_array):
    for col, val in enumerate(res):
        df.iat[idx, col] = val

df

,"(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, acidification: terrestrial, terrestrial acidification potential (TAP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, climate change, global warming potential (GWP1000))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, ecotoxicity: freshwater, freshwater ecotoxicity potential (FETP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, ecotoxicity: marine, marine ecotoxicity potential (METP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, ecotoxicity: terrestrial, terrestrial ecotoxicity potential (TETP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, energy resources: non-renewable, fossil, fossil fuel potential (FFP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, eutrophication: freshwater, freshwater eutrophication potential (FEP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, eutrophication: marine, marine eutrophication potential (MEP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, human toxicity: carcinogenic, human toxicity potential (HTPc))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, human toxicity: non-carcinogenic, human toxicity potential (HTPnc))",...,"(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, land use, agricultural land occupation (LOP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, material resources: metals/minerals, surplus ore potential (SOP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, ozone depletion, ozone depletion potential (ODPinfinite))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, particulate matter formation, particulate matter formation potential (PMFP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, photochemical oxidant formation: human health, photochemical oxidant formation potential: humans (HOFP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, photochemical oxidant formation: terrestrial ecosystems, photochemical oxidant formation potential: ecosystems (EOFP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, water use, water consumption potential (WCP))","(ReCiPe 2016 v1.03, endpoint (H) - no biogenic, total: ecosystem quality, ecosystem quality)","(ReCiPe 2016 v1.03, endpoint (H) - no biogenic, total: human health, human health)","(ReCiPe 2016 v1.03, endpoint (H) - no biogenic, total: natural resources, natural resources)"
production of alchohol wipes,0.000007,0.00203,0.000067,0.00009,0.006561,0.000908,0.000001,0.0,0.00009,0.001907,...,0.000714,0.000623,0.0,0.000003,0.000005,0.000006,0.000031,0.0,0.0,0.000462
packaging of glass vials with penicillin G,0.000512,0.169821,0.008423,0.011072,0.542653,0.045054,0.000083,0.00002,0.008409,0.18881,...,0.018216,0.002196,0.0,0.000306,0.000398,0.000413,0.001349,0.0,0.0,0.011108
production of a pair of gloves,0.000514,0.146823,0.004967,0.00668,0.42594,0.081017,0.000042,0.000006,0.006937,0.122456,...,0.015306,0.002288,0.0,0.000224,0.000444,0.000479,0.001755,0.0,0.0,0.031331
"treatment of hazardous waste, hazardous waste incineration, with energy recovery",0.000195,0.237005,0.003999,0.005457,0.218445,0.023689,0.000065,0.000005,0.025994,0.093784,...,0.000942,0.003721,0.0,0.000089,0.000176,0.000188,0.000804,0.0,0.0,0.009756
production of IV bag,0.000578,0.202636,0.006016,0.007916,0.558383,0.095157,0.000051,0.000007,0.009045,0.133127,...,0.006223,0.004886,0.0,0.000265,0.000435,0.000462,0.002272,0.0,0.0,0.034837
medical connector,0.000547,0.11141,0.025321,0.037464,0.913683,0.032238,0.000063,0.000004,0.017701,0.445623,...,0.002402,0.00366,0.0,0.000236,0.00033,0.000339,0.001089,0.0,0.0,0.00956
"market for sodium chlorate, powder",0.000007,0.002039,0.000162,0.000213,0.007936,0.000547,0.000002,0.0,0.000388,0.003697,...,0.000065,0.000122,0.0,0.000003,0.000004,0.000004,0.000108,0.0,0.0,0.000145
stopcock,0.000094,0.032432,0.001107,0.001428,0.102507,0.01433,0.000009,0.000001,0.001539,0.02374,...,0.000704,0.000555,0.0,0.000045,0.000072,0.000077,0.000368,0.0,0.0,0.005007
"market for water, ultrapure",0.000001,0.000261,0.000011,0.000015,0.000554,0.000067,0.000005,0.000001,0

In [30]:
df = lr.quick_LCIA("penicillin")
df[df.columns[1]]

Initialization is completed for penicillin_cut_off_IN


Penicillin G, defined system    {''production of alchohol wipes' (unit, DK, No...
Penicillin V, defined system    {''packaging of a medicine strip' (unit, IN, N...
Name: (ReCiPe 2016 v1.03, midpoint (H) - no biogenic, climate change, global warming potential (GWP1000)), dtype: object

In [35]:
for dct in df[df.columns[1]]:
    for key, val in dct.items():
        print(key, val)

'production of alchohol wipes' (unit, DK, None) 2.2658808957067293
'packaging of glass vials with penicillin G' (unit, IN, None) 0.000260818037913884
'production of a pair of gloves' (unit, MY, None) 0.11140952911663385
'treatment of hazardous waste, hazardous waste incineration, with energy recovery' (kilogram, CH, None) -0.12108993114785323
'production of IV bag' (unit, RER, None) 1.698214855368787
'medical connector' (unit, GLO, None) 0.20263606509307353
'market for sodium chlorate, powder' (kilogram, RER, None) 6.545501480006091e-06
'stopcock' (unit, GLO, None) 0.14682347704705836
'market for water, ultrapure' (kilogram, RER, None) 0.00020304652313836923
'waste packaging paper, Recycled Content cut-off' (kilogram, GLO, None) 0.000655253164342518
'packaging of a medicine strip' (unit, IN, None) 0.07552936319022431
'production of a medicin cup' (unit, RER, None) 0.00260818037913884
'treatment of hazardous waste, hazardous waste incineration' (kilogram, CH, None) -0.000278189594204234